In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3
EPOCHS=50

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "Dataset for Android App",
    shuffle = True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE

)

Found 2361 files belonging to 9 classes.


In [4]:
class_names = dataset.class_names
class_names

['Local Sindhi Green',
 'Local Sindhi Mature Green',
 'Local Sindhi Ripe',
 'Riyali Green',
 'Riyali Mature Green',
 'Riyali Ripe',
 'Thadhrami Green',
 'Thadhrami Mature Green',
 'Thadhrami Ripe']

In [5]:
class_names =["Local Sindhi Green", "Local Sindhi Mature Green", "Local Sindhi Ripe","Riyali Green","Riyali Mature Green","Riyali Ripe","Thadhrami Green","Thadhrami Mature Green","Thadhrami Ripe"]

In [6]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [7]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [10]:
len(train_ds)

59

In [11]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 9

model = models.Sequential([
    tf.keras.layers.Rescaling(1./255),
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes)
])

model.build(input_shape=input_shape)

In [12]:
model.compile(
    optimizer="adam",
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics=['accuracy']
)

In [13]:
model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    epochs=50,
)

Epoch 1/50
59/59 [==============================] - 51s 712ms/step - loss: 1.9694 - accuracy: 0.1999 - val_loss: 1.9224 - val_accuracy: 0.2232
Epoch 2/50
59/59 [==============================] - 43s 672ms/step - loss: 1.7747 - accuracy: 0.2934 - val_loss: 1.3963 - val_accuracy: 0.4464
Epoch 3/50
59/59 [==============================] - 41s 678ms/step - loss: 1.3362 - accuracy: 0.4567 - val_loss: 1.1587 - val_accuracy: 0.5357
Epoch 4/50
59/59 [==============================] - 39s 651ms/step - loss: 1.0869 - accuracy: 0.5731 - val_loss: 0.9441 - val_accuracy: 0.6696
Epoch 5/50
59/59 [==============================] - 40s 658ms/step - loss: 0.8914 - accuracy: 0.6396 - val_loss: 0.5774 - val_accuracy: 0.7812
Epoch 6/50
59/59 [==============================] - 40s 658ms/step - loss: 0.6992 - accuracy: 0.7225 - val_loss: 0.5886 - val_accuracy: 0.8125
Epoch 7/50
59/59 [==============================] - 39s 651ms/step - loss: 0.6208 - accuracy: 0.7560 - val_loss: 0.4914 - val_accuracy: 0.8125

In [14]:
model.evaluate(test_ds)

8/8 [==============================] - 11s 212ms/step - loss: 0.0600 - accuracy: 0.9727


[0.059960972517728806, 0.97265625]

In [15]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("C:/Users/Abdul Aziz Kehar/OneDrive/Desktop/tflite model/Model.tflite",'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\ABDULA~1\AppData\Local\Temp\tmp5f5w20aw\assets
